# 7) Testing Linear Regression - Prudential Life

## Importing Packages and loading into DF

In [2]:
from __future__ import print_function
import timeit

from sklearn.model_selection import learning_curve, ShuffleSplit
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

%matplotlib inline

import xgboost as xgb
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import log_loss, confusion_matrix, mean_squared_error

pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',500)

In [3]:
ins_data = './dataset/train.csv'
ins = pd.read_csv(ins_data)
# ins_test_data = './dataset/test.csv'
# ins_test = pd.read_csv(ins_test_data)

In [4]:
print(ins.shape)
# print(ins_test.shape)

(59381, 128)


## Cleaing Data
- Dropping Axis Column
- Creating dummy for column Product_Info_2 column
- Removing NaN values - where normally distributed, replacing with mean value, where other columns used meadian.

In [5]:
ins.drop('Id',axis=1,inplace=True)
# ins_test.drop('Id',axis=1,inplace=True)

In [6]:
ins.Product_Info_2.value_counts()
prod_2 = {'A1':1,'A2':2,'A3':3,'A4':4,'A5':5,'A6':6,'A7':7,'A8':8,'B1':9,'B2':10,'C1':11,'C2':12,
          'C3':13,'C4':14,'D1':15,'D2':16,'D3':17,'D4':18,'E1':19}

ins.replace({"Product_Info_2": prod_2},inplace=True)

In [7]:
for i in range(2,6):
    ins['Family_Hist_%i' % i].fillna(0,inplace=True)
#     ins_test['Family_Hist_%i' % i].fillna(0,inplace=True)

for i in [1,10,15,24,32]:
    ins['Medical_History_%i' % i].fillna(0,inplace=True)
#     ins_test['Medical_History_%i' % i].fillna(0,inplace=True)

for i in [1,4,6]:
    ins['Employment_Info_%i' % i].fillna(0,inplace=True)
#     ins_test['Employment_Info_%i' % i].fillna(0,inplace=True)

ins['Insurance_History_5'].fillna(0,inplace=True)
# ins_test['Insurance_History_5'].fillna(0,inplace=True)

In [8]:
ins.isna().sum()

Product_Info_1         0
Product_Info_2         0
Product_Info_3         0
Product_Info_4         0
Product_Info_5         0
Product_Info_6         0
Product_Info_7         0
Ins_Age                0
Ht                     0
Wt                     0
BMI                    0
Employment_Info_1      0
Employment_Info_2      0
Employment_Info_3      0
Employment_Info_4      0
Employment_Info_5      0
Employment_Info_6      0
InsuredInfo_1          0
InsuredInfo_2          0
InsuredInfo_3          0
InsuredInfo_4          0
InsuredInfo_5          0
InsuredInfo_6          0
InsuredInfo_7          0
Insurance_History_1    0
Insurance_History_2    0
Insurance_History_3    0
Insurance_History_4    0
Insurance_History_5    0
Insurance_History_7    0
Insurance_History_8    0
Insurance_History_9    0
Family_Hist_1          0
Family_Hist_2          0
Family_Hist_3          0
Family_Hist_4          0
Family_Hist_5          0
Medical_History_1      0
Medical_History_2      0
Medical_History_3      0


In [9]:
mvc = ['Family_Hist_2','Family_Hist_3','Family_Hist_4','Family_Hist_5','Medical_History_1',
       'Medical_History_10','Medical_History_15','Medical_History_24','Medical_History_32',
       'Employment_Info_1','Employment_Info_4','Employment_Info_6','Insurance_History_5']
data = ins[pd.notnull(ins['Family_Hist_2'])]
target = ins[pd.isnull(ins['Family_Hist_2'])]

y = data['Family_Hist_2']
X = data.drop(mvc,axis=1)


## Linear Regression

In [9]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
scores = cross_val_score(reg,X,y,cv=5)

In [10]:
scores.mean()

0.12965417145379787

In [11]:
reg.fit(X,y)
predictions = reg.predict(target.drop(mvc,axis=1))
target['Family_Hist_2'] = predictions
ins = pd.concat([data,target])

ValueError: Found array with 0 sample(s) (shape=(0, 114)) while a minimum of 1 is required.

In [ ]:
ins.head()

In [ ]:
Family_Hist_3,Family_Hist_5,Medical_History_1

In [ ]:
mvc = ['Family_Hist_3','Family_Hist_4','Family_Hist_5','Medical_History_1',
       'Medical_History_10','Medical_History_15','Medical_History_24','Medical_History_32',
       'Employment_Info_1','Employment_Info_4','Employment_Info_6','Insurance_History_5']
data = ins[pd.notnull(ins['Family_Hist_4'])]
target = ins[pd.isnull(ins['Family_Hist_4'])]

y = data['Family_Hist_4']
X = data.drop(mvc,axis=1)
reg = LinearRegression()
scores = cross_val_score(reg,X,y,cv=5)
scores.mean()

In [ ]:
reg.fit(X,y)
predictions = reg.predict(target.drop(mvc,axis=1))
target['Family_Hist_4'] = predictions
ins = pd.concat([data,target])

In [ ]:
ins = ins[ins['Employment_Info_1'].notnull()]

## Attempting to fit polynomial with degree 2

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree=2)
x_poly = poly_reg.fit_transform(X)

In [ ]:
from sklearn import neighbors

mvc = ['Family_Hist_3','Family_Hist_5','Medical_History_1',
       'Medical_History_10','Medical_History_15','Medical_History_24','Medical_History_32',
       'Employment_Info_4','Employment_Info_6','Insurance_History_5']
data = ins[pd.notnull(ins['Insurance_History_5'])]
target = ins[pd.isnull(ins['Insurance_History_5'])]

y = data['Insurance_History_5']
X = data.drop(mvc,axis=1)
knn = neighbors.KNeighborsRegressor(n_neighbors= 5)
scores = cross_val_score(reg,X,y,cv=5,n_jobs=-1)
scores.mean()

In [ ]:
reg.fit(X,y)
predictions = reg.predict(target.drop(mvc,axis=1))
target['Family_Hist_4'] = predictions
ins = pd.concat([data,target])

In [ ]:
ins['Family_Hist_3'].hist()

## Creating X and y for modelling

In [10]:
for i in ins.columns:
    if str(ins[i].dtypes) == 'int64':
        ins[i] = ins[i].astype('int8')
    else:
        ins[i] = ins[i].astype('float16')

In [11]:
features = ins.drop(['Response','Ht','Wt'],axis=1)
targets = ins.Response
targets = targets.map(lambda x: x-1)
X, X_holdout, y, y_holdout = train_test_split(features,targets, train_size = 0.8,test_size=0.2,random_state=77)
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.9,test_size=0.1,random_state=50)

#### Determining Baseline

In [12]:
y_train.value_counts()

7    14028
5     8069
6     5821
1     4714
0     4489
4     3906
3     1003
2      723
Name: Response, dtype: int64

In [13]:
baseline=15637/len(y_train)
print(baseline)

0.36575211096297333


## XGBoost

Attempting to use XGBoost Regressor

In [14]:
xgb = XGBRegressor(n_estimators=100,max_depth=6,subsample=0.8,min_child_weight=5,eta=0.02,verbosity= 1,colsample_bytree=0.6,objective= 'multi:softprob',num_class= 8,gamma=5,reg_lambda=10,reg_alpha=6)
xgb.fit(X_train.values,y_train)

C:\Users\Roy\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.6, eta=0.02, gamma=5,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=5, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, num_class=8, objective='multi:softprob',
       random_state=0, reg_alpha=6, reg_lambda=10, scale_pos_weight=1,
       seed=None, silent=None, subsample=0.8, verbosity=1)